### Importación de bibliotecas necesarias

In [1]:
import numpy as np

### Función 1: Dominancia de pareto para vectornes N-dimensionales

In [2]:
def dominancia_pareto(U, V, modo="minimize"):
    """
    Determina la relación de dominancia entre dos vectores U y V.
    
    Args:
    - U (list): Vector n-dimensional.
    - V (list): Vector n-dimensional (mismo tamaño que U).
    - modo (str): "minimize" o "maximize".
    
    Returns:
    - 1 si U domina a V.
    - 2 si V domina a U.
    - 3 si ninguno domina al otro.
    """
    if len(U) != len(V):
        raise ValueError("Los vectores U y V deben tener la misma longitud.")
    
    if modo not in ["minimize", "maximize"]:
        raise ValueError("El modo debe ser 'minimize' para minimización o 'maximize' para maximización.")
    
    if modo == "minimize":
        u_domina_v = all(u <= v for u, v in zip(U, V)) and any(u < v for u, v in zip(U, V))
        v_domina_u = all(v <= u for v, u in zip(V, U)) and any(v < u for v, u in zip(V, U))
    else:  
        u_domina_v = all(u >= v for u, v in zip(U, V)) and any(u > v for u, v in zip(U, V))
        v_domina_u = all(v >= u for v, u in zip(V, U)) and any(v > u for v, u in zip(V, U))
    
    if u_domina_v:
        return 1
    elif v_domina_u:
        return 2
    else:
        return 3

In [3]:
array1 = np.array([10, 20, 40, 80])
array2 = np.array([10, 20, 40, 80])

condicion = dominancia_pareto(array1, array2)

print( condicion)


3


### Función 2: Fast-non-dominated-sort

In [4]:
def fast_non_dominated_sort(vectores):
    """
    Ordena un conjunto de vectores en R^n de acuerdo a la dominancia de pareto

    Args:
        vectores (list): Un conjunto de vectores cuyas componentes son los valores del fitness para cada 
        uno de los objetivos

    Returns:
        _type_: _description_
    """
    frentes = [[]]
    soluciones_dominadas = [ ]
    cont_dominantes = [ ]
    
    # Se calcula s_p y n_p para cada uno de los cromosomas 
    for vector_pivote in vectores: # for each p in P
        s_p = []  # --->  soluciones a las que domina p
        n_p = 0   # --> # de soluciones que dominan a p  
        for vector_comparado in vectores: # for each q in P 
            ################# FALTA GESTIONAR CUANDO AMBOS VECTORES SON IGUALES O NO SE DOMINAN ENTRE SÍ
            comparacion = dominancia_pareto(vector_pivote, vector_comparado)
            if comparacion == 1:   
                s_p.append(vector_comparado)
            elif comparacion == 2:
                n_p += 1
            else:
                pass
        # Para encontrar el primer frente     
        if n_p == 0:
                p_rank = 1
                frentes[0].append(vector_pivote)
        
    i = 0 # El contador de frentes 0 = primer frente 
    while frentes[i]:
        print("EJECUTÁNDOSE")
        Q = [ ]
        for vector in frentes[i]:
             

                
            
                

    return None 

IndentationError: expected an indented block after 'for' statement on line 38 (4217573983.py, line 45)

In [ ]:
def fast_non_dominated_sort_2(vectores, modo="minimize"):
    """
    Ordena un conjunto de vectores en R^n de acuerdo a la dominancia de Pareto,
    siguiendo el algoritmo Fast Non-Dominated Sort de NSGA-II.

    Args:
        vectores (list): Una lista de vectores (listas de números).
        modo (str): "minimize" o "maximize", para pasarlo a la función de dominancia.

    Returns:
        list: Una lista de frentes. Cada frente es una lista de los vectores
              que pertenecen a ese nivel de no dominancia.
    """
    n_vectores = len(vectores)
    
    # --- Listas para almacenar S_p y n_p para cada vector ---
    # Usamos los índices para referenciar a los vectores
    soluciones_dominadas = [[] for _ in range(n_vectores)] # S_p para cada p
    contador_dominancia = [0] * n_vectores # n_p para cada p
    
    # --- Paso 1: Calcular S_p y n_p para cada individuo ---
    for i in range(n_vectores):
        for j in range(n_vectores):
            if i == j:
                continue
            
            relacion = dominancia_pareto(vectores[i], vectores[j], modo)
            
            if relacion == 1: # El vector i domina al vector j
                soluciones_dominadas[i].append(j)
            elif relacion == 2: # El vector j domina al vector i
                contador_dominancia[i] += 1
    
    # --- Paso 2: Identificar el primer frente (n_p = 0) ---
    frentes = [[]]
    for i in range(n_vectores):
        if contador_dominancia[i] == 0:
            frentes[0].append(i)
            
    # --- Paso 3: Construir los frentes sucesivos ---
    k = 0
    while frentes[k]:
        Q = [] # Almacenará los miembros del siguiente frente (k+1)
        
        # Para cada índice 'i' en el frente actual F_k
        for i in frentes[k]:
            # Para cada índice 'j' en el conjunto de soluciones dominadas por 'i' (S_p)
            for j in soluciones_dominadas[i]:
                contador_dominancia[j] -= 1
                if contador_dominancia[j] == 0:
                    Q.append(j)
        
        k += 1
        if Q:
            frentes.append(Q)
        else:
            break
            
    # --- Formatear la salida para devolver los vectores en lugar de los índices ---
    resultado_final = []
    for frente in frentes:
        vectores_frente = [vectores[i] for i in frente]
        resultado_final.append(vectores_frente)
        
    return resultado_final

### Funcion para codificar crowding-distance-assignment

In [ ]:
def crowding_distance_assignment(poblacion_soluciones):
    """Funcion que calcula la distancia de crowding 

    Args:
        poblacion_soluciones (list): conjunto de vectores donde cada componente es el fitness por cada objetivo de la solución

    Returns:
        sobrevivientes (list): Lista con las distancias de crowding para cada vector del conjunto de entrada
    """

    l = len(poblacion_soluciones) # El del arreglo de entrada
    num_obj = len(poblacion_soluciones[0]) # el numero de objetivos por vector 
    distancias = [0.0 for _ in range(l)] # array para almacenar las distancias de crowding 

    for i in range(num_obj):
        I_ordenada = sorted(poblacion_soluciones, key=lambda x:x[i]) # Lista ordenada 
        distancias[0] = distancias[l-1] = 100000
        rango = I_ordenada[-1][i] - I_ordenada[0][i] # Valores mínimos y máximos (f_max - f_min)
        for j in range(1,l-1):
            distancias[j] = distancias[j] + ((I_ordenada[j+1][i] - I_ordenada[j-1][i])/rango)
            
    return distancias

In [ ]:
## GENERANDO UNA SEGUNDA FUNCIÓN PARA UNA DISTANCIA DE CROWDING 

def crowding_distance_2 (poblacion_soluciones):

    l = len(poblacion_soluciones)
    num_obj = len(poblacion_soluciones[0])
    distancias = [0.0 for _ in range(l)]
    array_index = [i for i in range(l)]
    #print(array_index)

    for i in range(num_obj):
        I_ordenada = sorted(poblacion_soluciones, key= lambda x:x[i]) # Lista ordenadas con los vectores 
        indices_ordenados = sorted(range(l), key=lambda j:poblacion_soluciones[j][i])
        distancias[0] = distancias [-1] = 100000
        rango = I_ordenada[-1][i] - I_ordenada[0][i]  # Como se ordenan de menor a mayor, max está en la última posición y min en la primera
        #print("**************************************************************")
        #print(f"Para el objetivo {i+1} con el array ordenado {indices_ordenados} se esjecutan las siguientes operaciones")
        for j in range(1,l-1):
            idx = indices_ordenados[j]
            #print(f"--> distancias[{idx}] = distancias[{idx}] ({distancias[idx]}) + ((poblacion_soluciones[{idx+1}][{i}]({poblacion_soluciones[idx+1][i]}) - poblacion_soluciones[{idx+-1}][{i}]({poblacion_soluciones[idx+-1][i]}) / {rango}")
            distancias[idx] = distancias[idx] + (abs((poblacion_soluciones[idx+1][i] - poblacion_soluciones[idx-1][i]))/rango)
            #print(f"Dando commo resultado: {distancias[idx]}")
    return distancias

In [ ]:
## PROBANDO LA SEGUNDA FUNCION DE LA DISTANCIA DE CROWDING

import numpy as np
vectores_frente_2 = np.array([[2,11], [3,10], [4,7], [5,6], [7,5], [9,3], [10,2], [12,1]])
distancias = crowding_distance_2(vectores_frente_2)


indices_ordenados = sorted(range(len(vectores_frente_2)), key=lambda i:vectores_frente_2[i][1])
print(f"Las distancias son: {distancias}")


In [ ]:
def sobrevivientes_dist_crowding(vec_frente, n_sobrevivientes):
    """_summary_

    Args:
        vec_frente (list): Conjunto de vectores con los valores del fitness para cada objetivo dentro de frente de pareto correspondiente
                           para ajustar el tamaño de la población.
        n_sobrevivientes (int): Valor que indica cuáles soluciones sobrevivirán. 
    
    Returns:
        soluciones (list): Lista con los vectores de las soluciones que sobrevirián
        indices (list): Lista con los índices de las soluciones que sobrevivirán
    """
    distancias_crowding = crowding_distance_2(vec_frente)
    print(f"Las distancias de crowding para los vectores son: {distancias_crowding}")
    indices = sorted(range(len(distancias_crowding)), key=lambda i: distancias_crowding[i], reverse=True)[:n_sobrevivientes]
    soluciones = [vec_frente[i] for i in indices]

    return soluciones, indices

In [ ]:
sol, idxs = sobrevivientes_dist_crowding(vectores_frente_2, 5)
print(f"Las soluciones que sobreviven son: {sol}")
print(f"Los índices de las soluciones que sobreviven son: {idxs}")

### Función para evaluar el fitness de una solución

In [ ]:
# Cargando las funciones de un archio previamente adquirido 
from datos_transformacion import *

m_oper_energia = cargar_matriz_operaciones_maquina(r"C:\Users\DELL5400\Documents\noveno semestre\Topicos de Algoritmos Bioinspirados\Genetic-Algorithms\Problema asignación de tareas\clase\datos_prueba\datos_operacion_consumo.txt")
m_oper_tiempo = cargar_matriz_operaciones_maquina(r"C:\Users\DELL5400\Documents\noveno semestre\Topicos de Algoritmos Bioinspirados\Genetic-Algorithms\Problema asignación de tareas\clase\datos_prueba\datos_operacion_tiempo.txt")
tareas = cargar_matriz_tareas(r"C:\Users\DELL5400\Documents\noveno semestre\Topicos de Algoritmos Bioinspirados\Genetic-Algorithms\Problema asignación de tareas\clase\datos_prueba\tareas.txt")


print(f"\n\n La matriz operaciones-tiempo es: {m_oper_tiempo} y es de tipo {type(m_oper_tiempo)}")
print(f"\n\n La matriz operaciones-energía es: {m_oper_energia} y es de tipo {type(m_oper_energia)}")
print(f"\n\n La lista de tareas es: {tareas} y es de tipo {type(tareas)}")


In [ ]:
def evaluar_fitness(vec_solucion, mtx_op_t, mtx_op_e, l_tsk_oper):

    # Datos relevantes 
    num_tareas = len(l_tsk_oper)
    num_maquinas = len(mtx_op_t[0])

    # Hacer un cast a un vector de numpy 
    vec_solucion = np.array(vec_solucion)

    # Generar las listas para llevar el conteo del makespan
    tiemp_maquinas = [ [0.0] for _ in range(num_maquinas)]
    tiemp_tareas = [[0.0] for _ in range(num_tareas)]
    energia_consumida = 0.0
    makespan = 0.0

    # Recorriendo cada una de las operaciones por tarea 
    idx_op_maq = 0 # Es un índice que nos ayuda a rastrear la maquina en donde se ejecutará la operación 
    idx_ant = 0
    maquina_ant = 0
    duracion_ant = 0.0
    for i in range(len(l_tsk_oper)): # Recorriendo cada tarea
        for j in range(len(l_tsk_oper[i])): #Recorriendo la operación J de la tarea I 
            maquina = vec_solucion[idx_op_maq]
            operacion = l_tsk_oper[i][j]
            energia_consumida = energia_consumida + mtx_op_e[operacion-1][maquina-1] # Actualizacion energía
            print("------------------------------------------------------------")
            print(f"La actividad: {idx_op_maq+1} es la operación {operacion}, se ejecuta en la maquina {maquina} y consume {mtx_op_e[operacion-1][maquina-1]} energia")
            duracion = mtx_op_t[operacion-1][maquina-1]
            print(f"Los parámetros inciales son: Maquina = {maquina}, Operación= {operacion}, duracion = {duracion}")

            # Actualizando el makespan
            print(f"Act. tiempo de la tarea {i+1} con {tiemp_tareas[i][-1]} + {duracion}, ANTES = {tiemp_tareas[i]}")
            tiemp_tareas[i].append(tiemp_tareas[i][-1] + duracion)
            print(f"Luego de la actualización: {tiemp_tareas[i]} \n")
            # Implementando la secuencialidad de las operaciones 
            if idx_op_maq > 0 and idx_ant == i:
                print(f"\tEvalua V, la tarea anterior fue: {idx_ant } y la tarea actual es {i}")
                if maquina != maquina_ant:
                    print(f"\t\tEvalua V: la maquina anterior fue: {maquina_ant} y la maquina actual es: {maquina}")
                    print(f"\t\tAct. tiempo de la maquina {maquina} con {tiemp_maquinas[maquina-1][-1]} + {duracion_ant} (duracion de la tarea anterior), ANTES = {tiemp_maquinas[maquina-1]}")
                    tiemp_maquinas[maquina-1].append(tiemp_maquinas[maquina-1][-1] + duracion_ant)
                    print(f"\t\tLuego de la actualización: {tiemp_maquinas[maquina-1]} \n")
                else:
                    print(f"\t\tLa primera condicion fue V, pero la maquina anterior fue {maquina_ant} y la maquina actual es: {maquina}, son la misma máquina")
            else:
                print(f"\tEvalua F, pues la tarea anterior fue {idx_ant+1} y la tarea actual es: {i+1}")
            
            print(f"*Act. tiempo de la maquina {maquina} con {tiemp_maquinas[maquina-1][-1]} + {duracion} (duracion actual), ANTES = {tiemp_maquinas[maquina-1]}")
            tiemp_maquinas[maquina-1].append(tiemp_maquinas[maquina-1][-1] + duracion)
            print(f"Luego de la actualización: {tiemp_maquinas[maquina-1]} \n")

            makespan_local = max ( max(tiemp_tareas[i]), max(tiemp_maquinas[maquina-1]))
            print(f"mkpan local = max [{max(tiemp_tareas[i])} , {max(tiemp_maquinas[maquina-1])}] = {makespan_local}")


            if makespan < makespan_local:
                print(f"\tEvalua en V, pues el makespan local es mayor que el general, makespan = {makespan} < makespan local = {makespan_local}")
                makespan = makespan + duracion
                print(f"El makespan se actualizó a {makespan}")
            else:
                makespan = makespan
                print("El makespan se mantuvo")
            
            idx_op_maq += 1
            
            # Actualizando los valores auxiliares para la siguiente iteración
            idx_ant = i
            maquina_ant = maquina
            duracion_ant = duracion
            print(f"Los parámetros se actualizaron a: IOM = {idx_op_maq}, IA = {idx_ant}, MA = {maquina_ant}, DA = {duracion_ant}")


    return np.array([makespan, energia_consumida])

### PRUEBA 2 DE LA FUNCION PARA CALCULAR EL FITNESS

In [ ]:
def evaluar_fitness(vec_solucion, mtx_op_t, mtx_op_e, l_tsk_oper):

    print("Esta es la segunda funcion")
    # Datos relevantes 
    num_tareas = len(l_tsk_oper)
    num_maquinas = len(mtx_op_t[0])

    # Hacer un cast a un vector de numpy 
    vec_solucion = np.array(vec_solucion)

    # Generar las listas para llevar el conteo del makespan
    tiemp_maquinas = [ [0.0] for _ in range(num_maquinas)]
    tiemp_tareas = [[0.0] for _ in range(num_tareas)]
    energia_consumida = 0.0
    makespan = 0.0

    # Recorriendo cada una de las operaciones por tarea 
    idx_op_maq = 0 # Es un índice que nos ayuda a rastrear la maquina en donde se ejecutará la operación 
    idx_ant = 0
    maquina_ant = 0
    duracion_ant = 0.0
    for i in range(len(l_tsk_oper)): # Recorriendo cada tarea
        for j in range(len(l_tsk_oper[i])): #Recorriendo la operación J de la tarea I 
            maquina = vec_solucion[idx_op_maq]
            operacion = l_tsk_oper[i][j]
            energia_consumida = energia_consumida + mtx_op_e[operacion-1][maquina-1] # Actualizacion energía
            print("------------------------------------------------------------")
            print(f"La actividad: {idx_op_maq+1} es la operación {operacion}, se ejecuta en la maquina {maquina} y consume {mtx_op_e[operacion-1][maquina-1]} energia")
            duracion = mtx_op_t[operacion-1][maquina-1]
            print(f"Los parámetros inciales son: Maquina = {maquina}, Operación= {operacion}, duracion = {duracion}")

            # Actualizando el makespan
            print(f"Act. tiempo de la tarea {i+1} con {tiemp_tareas[i][-1]} + {duracion}, ANTES = {tiemp_tareas[i]}")
            tiemp_tareas[i].append(tiemp_tareas[i][-1] + duracion)
            print(f"Luego de la actualización: {tiemp_tareas[i]} \n")
            # Implementando la secuencialidad de las operaciones 
            if idx_op_maq > 0 and idx_ant == i:
                print(f"\tEvalua V, la tarea anterior fue: {idx_ant } y la tarea actual es {i}")
                if maquina != maquina_ant:
                    print(f"\t\tEvalua V: la maquina anterior fue: {maquina_ant} y la maquina actual es: {maquina}")
                    print(f"\t\tAct. tiempo de la maquina {maquina} con {tiemp_maquinas[maquina-1][-1]} + {duracion_ant} (duracion de la tarea anterior), ANTES = {tiemp_maquinas[maquina-1]}")
                    tiemp_maquinas[maquina-1].append(tiemp_maquinas[maquina-1][-1] + duracion_ant)
                    print(f"\t\tLuego de la actualización: {tiemp_maquinas[maquina-1]} \n")
                else:
                    print(f"\t\tLa primera condicion fue V, pero la maquina anterior fue {maquina_ant} y la maquina actual es: {maquina}, son la misma máquina")
            else:
                print(f"\tEvalua F, pues la tarea anterior fue {idx_ant+1} y la tarea actual es: {i+1}")
            
            # CUANDO YA NO NOS ENCONTRAMOS EN LA PRIMER ITERACION



            print(f"*Act. tiempo de la maquina {maquina} con {tiemp_maquinas[maquina-1][-1]} + {duracion} (duracion actual), ANTES = {tiemp_maquinas[maquina-1]}")
            tiemp_maquinas[maquina-1].append(tiemp_maquinas[maquina-1][-1] + duracion)
            print(f"Luego de la actualización: {tiemp_maquinas[maquina-1]} \n")

            makespan_local = max ( max(tiemp_tareas[i]), max(tiemp_maquinas[maquina-1]))
            print(f"mkpan local = max [{max(tiemp_tareas[i])} , {max(tiemp_maquinas[maquina-1])}] = {makespan_local}")


            if makespan < makespan_local:
                print(f"\tEvalua en V, pues el makespan local es mayor que el general, makespan = {makespan} < makespan local = {makespan_local}")
                makespan = makespan + duracion
                print(f"El makespan se actualizó a {makespan}")
            else:
                makespan = makespan
                print("El makespan se mantuvo")
            
            idx_op_maq += 1
            
            # Actualizando los valores auxiliares para la siguiente iteración
            idx_ant = i
            maquina_ant = maquina
            duracion_ant = duracion
            print(f"Los parámetros se actualizaron a: IOM = {idx_op_maq}, IA = {idx_ant}, MA = {maquina_ant}, DA = {duracion_ant}")


    return np.array([makespan, energia_consumida])

## PRUEBA 3 PARA CALCULAR EL MAKESPAN

In [ ]:
import numpy as np

def evaluar_fitness(vec_solucion, mtx_op_t, mtx_op_e, l_tsk_oper):
    print("ESTA ES LA TERCERA FUNCION")

    # --- CAMBIO 1: Simplificar los contadores de tiempo ---
    # Ahora son listas simples que actúan como "relojes".
    # Guardan el TIEMPO DE FINALIZACIÓN, no duraciones acumuladas.
    num_tareas = len(l_tsk_oper)
    num_maquinas = len(mtx_op_t[0])
    tiemp_maquinas = [0.0] * num_maquinas  # Reloj de cada máquina
    tiemp_tareas = [0.0] * num_tareas      # Reloj de cada tarea

    energia_consumida = 0.0
    
    # Hacer un cast a un vector de numpy 
    vec_solucion = np.array(vec_solucion)

    # Recorriendo cada una de las operaciones por tarea 
    idx_op_maq = 0 # Es un índice que nos ayuda a rastrear la operacion en vec_solucion
    for i in range(len(l_tsk_oper)): # Recorriendo cada tarea
        for j in range(len(l_tsk_oper[i])): # Recorriendo la operación J de la tarea I 
            maquina = vec_solucion[idx_op_maq]
            operacion = l_tsk_oper[i][j]
            
            energia_consumida += mtx_op_e[operacion-1][maquina-1] 
            duracion = mtx_op_t[operacion-1][maquina-1] # Obtener la duracion de la operacion en proceso 
            
            print("------------------------------------------------------------")
            print(f"Tarea {i+1}, Operación {operacion}: asignada a Máquina {maquina} (dura {duracion}, consume {mtx_op_e[operacion-1][maquina-1]})")

            # --- CAMBIO 2: La Lógica Clave del Makespan ---
            # La operación actual NO PUEDE empezar hasta que:
            # 1. La máquina esté libre.
            # 2. La operación anterior de ESTA MISMA TAREA haya terminado.
            
            tiempo_fin_op_anterior = tiemp_tareas[i]
            tiempo_maquina_libre = tiemp_maquinas[maquina-1]
            
            print(f"\tLa tarea {i+1} está lista desde t={tiempo_fin_op_anterior}")
            print(f"\tLa máquina {maquina} está libre desde t={tiempo_maquina_libre}")

            # El tiempo de inicio es el MÁXIMO de los dos anteriores.
            tiempo_inicio = max(tiempo_fin_op_anterior, tiempo_maquina_libre)
            
            # El tiempo de fin de la operación actual
            tiempo_fin_actual = tiempo_inicio + duracion
            
            print(f"\t==> La operación INICIA en t={tiempo_inicio:.2f} y TERMINA en t={tiempo_fin_actual:.2f}")

            # --- CAMBIO 3: Actualizar los "relojes" ---
            # Actualizamos el reloj de la máquina y de la tarea con el tiempo de finalización.
            tiemp_maquinas[maquina-1] = tiempo_fin_actual
            tiemp_tareas[i] = tiempo_fin_actual
            
            print(f"\tNuevo tiempo libre para Máquina {maquina}: {tiemp_maquinas[maquina-1]:.2f}")
            print(f"\tNuevo tiempo de finalización para Tarea {i+1}: {tiemp_tareas[i]:.2f}\n")
            
            idx_op_maq += 1

    # --- CAMBIO 4: Cálculo Final del Makespan ---
    # El makespan es el tiempo en que la ÚLTIMA máquina termina.
    # Es simplemente el valor máximo en nuestro reloj de máquinas.
    makespan = max(tiemp_maquinas)
    
    print("============================================================")
    print(f"TIEMPOS FINALES DE CADA MÁQUINA: {tiemp_maquinas}")
    print(f"Makespan Total de la Solución: {makespan:.2f}")
    print(f"Energía Total Consumida: {energia_consumida:.2f}")
    print("============================================================")

    return np.array([makespan, energia_consumida])

In [ ]:
m_oper_energia = cargar_matriz_operaciones_maquina(r"C:\Users\DELL5400\Documents\noveno semestre\Topicos de Algoritmos Bioinspirados\Genetic-Algorithms\Problema asignación de tareas\clase\datos_prueba\datos_operacion_consumo.txt")
m_oper_tiempo = cargar_matriz_operaciones_maquina(r"C:\Users\DELL5400\Documents\noveno semestre\Topicos de Algoritmos Bioinspirados\Genetic-Algorithms\Problema asignación de tareas\clase\datos_prueba\datos_operacion_tiempo.txt")
tareas = cargar_matriz_tareas(r"C:\Users\DELL5400\Documents\noveno semestre\Topicos de Algoritmos Bioinspirados\Genetic-Algorithms\Problema asignación de tareas\clase\datos_prueba\tareas.txt")

print(tareas)
vector_sol_prueba = np.array([1,2,3,2,1,3,1,2])

fitness = evaluar_fitness(vector_sol_prueba, m_oper_tiempo, m_oper_energia, tareas)
print(fitness)

In [ ]:
prueba = [0*0] * 5
print(prueba)

In [ ]:
vectores_prueba = np.array([[1,8], [2,11], [2,5], [3,10], [3,3], [4,7],
                            [5,11], [5,6], [6,2], [7,12], [7,5], [9,8], 
                            [9,3], [9,1], [10,2], [11,6], [12,1], [13,6]])


frentes_P = fast_non_dominated_sort_2(vectores_prueba)

for i in range(len(frentes_P)):
    print(f"El frente {i+1} tiene los vectores: {frentes_P[i]}")


## Obteniendo la distancia de crowding 

tam_poblacion = 10
print("------ DISTANCIA DE CROWDING-----")
sobrevivientes, indices = sobrevivientes_dist_crowding(frentes_P[1], tam_poblacion - len(frentes_P[0]))
print(f"Los sobrevivientes son: {sobrevivientes}")
print(f"Las soluciones que sobreviven son: {indices}")

print("************ EN EL EJERCICIO DE LA TAREA, ESTAS SOLUCIONES DEBEN DE ESTAR NUMERADAS")

In [ ]:
def fast_non_dominated_sort_con_indices(vectores, modo="minimize"):
    """
    Ordena un conjunto de vectores de acuerdo a la dominancia de Pareto (NSGA-II).

    Args:
        vectores (list): Una lista de vectores de fitness.
        modo (str): "minimize" o "maximize".

    Returns:
        tuple: Una tupla con dos elementos:
               1. (list): La lista de frentes con los VECTORES de fitness.
               2. (list): La lista de frentes con los ÍNDICES originales de esos vectores.
    """
    n_vectores = len(vectores)
    
    soluciones_dominadas = [[] for _ in range(n_vectores)]
    contador_dominancia = [0] * n_vectores
    
    # --- Paso 1: Calcular S_p y n_p ---
    for i in range(n_vectores):
        for j in range(n_vectores):
            if i == j:
                continue
            
            # Asumo que tienes la función dominancia_pareto definida en otra parte
            relacion = dominancia_pareto(vectores[i], vectores[j], modo)
            
            if relacion == 1: # i domina a j
                soluciones_dominadas[i].append(j)
            elif relacion == 2: # j domina a i
                contador_dominancia[i] += 1
    
    # --- Paso 2: Identificar el primer frente (n_p = 0) ---
    # Esta variable ahora contendrá los frentes con los índices
    frentes_con_indices = [[]]
    for i in range(n_vectores):
        if contador_dominancia[i] == 0:
            frentes_con_indices[0].append(i)
            
    # --- Paso 3: Construir los frentes sucesivos ---
    k = 0
    while frentes_con_indices[k]:
        Q = []
        for i in frentes_con_indices[k]:
            for j in soluciones_dominadas[i]:
                contador_dominancia[j] -= 1
                if contador_dominancia[j] == 0:
                    Q.append(j)
        
        k += 1
        if Q:
            frentes_con_indices.append(Q)
        else:
            break
            
    # --- Paso 4: Formatear la salida para tener AMBOS resultados ---
    # 1. Crear la lista de frentes con los vectores (como en la versión original)
    frentes_con_vectores = []
    for frente_indices in frentes_con_indices:
        vectores_frente = [vectores[i] for i in frente_indices]
        frentes_con_vectores.append(vectores_frente)
        
    # 2. Retornar ambas listas: la de vectores y la de índices
    return frentes_con_vectores, frentes_con_indices

## TERCERA VERSIÓN DE UNA FUNCIÓN PARA CALCULAR EL CROWDING DISTANCE 

In [ ]:
import numpy as np # Necesario si usas arrays de numpy

def crowding_distance_3(poblacion_soluciones):
    """
    Calcula la distancia de crowding para un conjunto de soluciones en un frente.
    Esta versión está corregida para evitar errores de índice.
    """
    # Convierte a lista de listas si son arrays de numpy, para consistencia
    poblacion_soluciones = [list(sol) for sol in poblacion_soluciones]

    l = len(poblacion_soluciones)
    if l <= 2:
        # Si hay 2 o menos soluciones, todas son extremos y su distancia es infinita
        return [float('inf')] * l

    num_obj = len(poblacion_soluciones[0])
    distancias = [0.0 for _ in range(l)]

    for i in range(num_obj):
        # Ordenamos los índices de las soluciones según el objetivo 'i'
        indices_ordenados = sorted(range(l), key=lambda j: poblacion_soluciones[j][i])
        
        # Asignamos una distancia infinita a las soluciones extremas para este objetivo
        # Esto asegura que los puntos en los bordes del frente de Pareto sean preferidos
        distancias[indices_ordenados[0]] = 10000
        distancias[indices_ordenados[-1]] = 10000

        # Obtenemos los valores mínimo y máximo para normalizar
        f_min = poblacion_soluciones[indices_ordenados[0]][i]
        f_max = poblacion_soluciones[indices_ordenados[-1]][i]
        rango = f_max - f_min

        # Si el rango es cero, no se puede calcular la distancia para este objetivo
        if rango == 0:
            continue

        # Calculamos la distancia para las soluciones intermedias
        for j in range(1, l - 1):
            # Índice original del individuo actual en la lista ordenada
            idx_original = indices_ordenados[j]
            
            # Valor del objetivo del vecino anterior en la lista ordenada
            valor_anterior = poblacion_soluciones[indices_ordenados[j-1]][i]

            # Valor del objetivo del vecino siguiente en la lista ordenada
            valor_siguiente = poblacion_soluciones[indices_ordenados[j+1]][i]
            
            # Acumulamos la distancia normalizada
            distancias[idx_original] += (valor_siguiente - valor_anterior) / rango

    return distancias

In [ ]:
def sobrevivientes_dist_crowding_3(vec_frente, n_sobrevivientes):
    """_summary_

    Args:
        vec_frente (list): Conjunto de vectores con los valores del fitness para cada objetivo dentro de frente de pareto correspondiente
                           para ajustar el tamaño de la población.
        n_sobrevivientes (int): Valor que indica cuáles soluciones sobrevivirán. 
    
    Returns:
        soluciones (list): Lista con los vectores de las soluciones que sobrevirián
        indices (list): Lista con los índices de las soluciones que sobrevivirán
    """
    distancias_crowding = crowding_distance_3(vec_frente)
    print(f"Las distancias de crowding para los vectores son: {distancias_crowding}")
    indices = sorted(range(len(distancias_crowding)), key=lambda i: distancias_crowding[i], reverse=True)[:n_sobrevivientes]
    soluciones = [vec_frente[i] for i in indices]

    return soluciones, indices

In [ ]:
vectores_prueba = np.array([[1,8], [2,11], [2,5], [3,10], [3,3], [4,7],
                            [5,11], [5,6], [6,2], [7,12], [7,5], [9,8], 
                            [9,3], [9,1], [10,2], [11,6], [12,1], [13,6]])


frentes_P, frentes_P_idx = fast_non_dominated_sort_con_indices(vectores_prueba)
print("----------------------")
print(frentes_P)
print("----------------")

for i in range(len(frentes_P)):
    print(f"El frente {i+1} tiene los vectores: {frentes_P[i]} y los índices {frentes_P_idx}")


## Obteniendo la distancia de crowding 

tam_poblacion = 10
print("------ DISTANCIA DE CROWDING-----")
sobrevivientes, indices = sobrevivientes_dist_crowding(frentes_P[1], tam_poblacion - len(frentes_P[0]))
print(f"Los sobrevivientes son: {sobrevivientes}")
print(f"Las soluciones que sobreviven son: {indices}")

print("************ AHORA PROBANDO LA SEGUNDA FUNCION PARA CALCULAR EL CROWDING DISTANCE *******")
sobrevivientes_2, indices_2 = sobrevivientes_dist_crowding_3(frentes_P[1], tam_poblacion-len(frentes_P[0]))
print(f"Los sobrevivientes son: {sobrevivientes_2}")
print(f"Las soluciones que sobreviven son: {indices_2}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tus datos de ejemplo
# (Uso np.array para que sea fácilmente reproducible)
datos = [
    [np.array([1, 8]), np.array([2, 5]), np.array([3, 3]), np.array([6, 2]), np.array([9, 1])],
    [np.array([2, 11]), np.array([3, 10]), np.array([4, 7]), np.array([5, 6]), np.array([7, 5]), np.array([9, 3]), np.array([10, 2]), np.array([12, 1])],
    [np.array([5, 11]), np.array([9, 8]), np.array([11, 6])],
    [np.array([7, 12]), np.array([13, 6])]
]

# --- INICIO DEL CÓDIGO PARA GRAFICAR ---

# 1. Crear la figura y los ejes
plt.figure(figsize=(10, 7))

# 2. Iterar sobre cada frente de Pareto en tu lista de datos
# Usamos enumerate para tener un índice (0, 1, 2...) para las leyendas
for i, frente in enumerate(datos):
    
    # TRUCO CLAVE: Ordenar los puntos por el valor de F1 (eje x)
    # Esto asegura que la línea que los une no se cruce y dibuje una curva limpia.
    frente_ordenado = sorted(frente, key=lambda p: p[0])
    
    # 3. Extraer las coordenadas X (F1) y Y (F2) del frente ordenado
    # Se usa una "list comprehension", una forma rápida de crear listas en Python.
    x_coords = [punto[0] for punto in frente_ordenado]
    y_coords = [punto[1] for punto in frente_ordenado]
    
    # 4. Graficar los puntos y la línea que los une
    # Matplotlib asignará un color diferente en cada iteración del bucle.
    plt.plot(
        x_coords, 
        y_coords, 
        marker='o',          # Estilo del punto (círculo)
        linestyle='-',       # Estilo de la línea (sólida)
        linewidth=0.5,       # Grosor de la línea (delgada)
        label=f'Frente {i+1}' # Etiqueta para la leyenda
    )

# 5. Añadir detalles finales a la gráfica
plt.title('Frentes de Pareto')
plt.xlabel('Función Objetivo 1 (F1)')
plt.ylabel('Función Objetivo 2 (F2)')
plt.legend() # Muestra la leyenda con las etiquetas de cada frente
plt.grid(True, linestyle='--', alpha=0.6) # Añade una rejilla suave

# 6. Mostrar la gráfica
plt.show()

In [ ]:
### Haciendo una función para la segunda distancia de crowding
def dist_crowding_2():
    return None

# PRUEBAS GENERALES 

In [ ]:
lista_gen = [[]]
vector = [1,2,4]

lista_gen[0].append(vector)

print(f"Lista general luego de la inserción: {lista_gen}")

In [ ]:
lista_t_2 = [4 ]

print("Lista no vacia" if lista_t_2 else "La lista esta vacía")

counter = 1

while lista_t_2 and counter < 5:
    print("La lista no se encuentra vacía")
    counter += 1

In [ ]:
arr_np = np.array([[3,4], [6,7], [2,1], [7,9]])
print(f"El tipo de array antes de la asignacion es: {type(arr_np)}")
lista = list(arr_np)
print(f"La última sublista de la lista es: {lista[-1]}")

lista_ordenada = sorted(lista, key=lambda x:x[0])
print(f"La lista ordenada por el primer elemento es {lista_ordenada}")
print(f"El valor máximo para el primer objetivo es: {lista_ordenada[-1][0]}")
print(f"El valor mínimo para el primer objetivo es: {lista_ordenada[0][0]}")
print(f"Una generacion de 8 es: {range(8)}")
array_3 = range(8)
print(array_3)

# PARA VERIFICAR QUE EL ALGORITMO ESTÁ CONVERGIENDO DESPUÉS DE UNA CIERTA CANTIDAD DE OPERACIONES EL NUMERO DE SOLUCIONES NO DOMINADAS DEBE DE AUMENTAR

# HAY QUE REPORTAR LAS SOLUCIONES EXTREMO (PRIMERO Y ÚLTIMO) Y ALGÚN PUNTO DEL CENTRO COMO SOLUCIONES FINALES

In [ ]:

lista_aleatoria= [1,2,3,4,5,6,7,8,9,0,11]

for i in range(len(lista_aleatoria)):
    if i < 5:
        print(i)
    else:
        print("Se rompio el ciclo")
        break

In [ ]:
array2 = [1,2,3,4,5,6,7, 9, 10]
for i in range(10):
    if i in [2,3,4]:
        print(i)

print(f"Se encontró" if i in array2 else "No hay nada")